In [ ]:
# Second meeting about sherlock
# Initial look at extraction and prediction performance on original gittables benchmark. 
# While also analysing some of the results.
# Additional comparison against sherlock dataset performance can be found in gittbables_sherlock_compare notebook

In [28]:
#Reload modules before executing code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from datetime import datetime
from os.path import join
from os import listdir
import json
import re #for camel case conversion


import numpy as np
import pandas as pd
from pyarrow.parquet import ParquetFile
from sklearn.metrics import f1_score, classification_report

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
# from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
#     convert_string_lists_to_lists,
    prepare_feature_extraction,
#     load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

### Utils

In [3]:
def camel_case(s):
  s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].lower(), s[1:]])

# def gittable_normalize(s):
#     return re.sub(r"[_-]","", " ".join(
#                 re.findall("[0-9,a-z,.,\"#!$%\^&\*;:{}=\-_`~()\n\t\d]+|[A-Z](?:[A-Z]*(?![a-z])|[a-z]*)", s)
#             )).lower()

In [4]:
print(camel_case('Team Name'))
print(camel_case('Team_name'))
# print(gittable_normalize('Team-Name99'))
# print(gittable_normalize('Team-name'))


teamName
teamName


## Loading the annotation data
#### Make sure to unzip tables.zip in the /data/data/gittables_benchmark directory before proceeding

In [5]:
path = '../data/data/gittables_benchmark'

with open(join(path, '../../types.json'), 'r') as f:  
        types = json.load(f)['type78']

# ground truth column annotations per table ID, 1 file per ontology: columns are “table_id” 
# (ignore the _<ontology> suffix), “target_column”, “annotation_id”, “annotation_label”.
dbpedia_gt = pd.read_csv(join(path, 'dbpedia_gt.csv'))
# unique labels present in the annotated tables, 1 file per ontology: 
# columns are “annotation_id” and “annotation_label”.
dbpedia_labels = pd.read_csv(join(path, 'dbpedia_labels.csv'))
# target columns per table ID, 1 file per ontology (DBpedia or Schema.org): 
# columns are “table_id” (ignore the "_<ontology>" suffix) and “target_column” 
# (i.e. the column that should be annotated).
dbpedia_targets = pd.read_csv(join(path, 'dbpedia_targets.csv'))
schema_gt = pd.read_csv(join(path, 'schema_gt.csv'))
schema_labels = pd.read_csv(join(path, 'schema_labels.csv'))
schema_targets = pd.read_csv(join(path, 'schema_targets.csv'))

test_table = pd.read_csv(join(path, 'tables/GitTables_1501.csv'), dtype='object')
test_table2 = pd.read_csv(join(path, 'tables/GitTables_1501.csv'))

In [6]:
# dbpedia_gt.head(10)

In [7]:
# dbpedia_labels.head()

In [8]:
# dbpedia_targets.head(5)

In [47]:
# test_table

In [48]:
# test_table2

### How many types from sherlock are in the test dataset?

In [6]:
type_matches = 0
testdata_labels = pd.concat([dbpedia_labels, schema_labels])
for gittable_type in map(camel_case,testdata_labels['annotation_label'].unique().tolist()):
    if gittable_type in types:
        type_matches+=1
#     else:
#         print(gittable_type)
print(type_matches)

39


In [10]:
columns = []
col_types = []
col_ids = []
filepaths = [join(path, 'tables/', f) for f in listdir(join(path, 'tables/')) if f.endswith('.csv')]

#Go over the tables in the dataset
for fp in filepaths:
    table_id = fp[fp.rfind('/')+1:-4]
    table_df = pd.read_csv(fp, dtype='object')
    dbpedia_table_gts = dbpedia_gt[dbpedia_gt['table_id'].str.contains(table_id)]
    dbpedia_table_gts = dbpedia_table_gts[['table_id', 'target_column', 'annotation_label']]
    dbpedia_table_gts['table_id'] = dbpedia_table_gts['table_id'].apply(lambda x: table_id)
    schema_table_gts = schema_gt[schema_gt['table_id'].str.contains(table_id)]
    schema_table_gts = schema_table_gts[['table_id', 'target_column', 'annotation_label']]
    schema_table_gts['table_id'] = schema_table_gts['table_id'].apply(lambda x: table_id)
    table_gts = pd.merge(dbpedia_table_gts, schema_table_gts, how='outer')
    
    for index, gt_row in table_gts.iterrows():
        col_type = camel_case(gt_row['annotation_label'])
        # We only want types that are used in sherlock
        if col_type not in types:
            continue
        col_name = 'col'+ str(gt_row['target_column'])
        col_id = table_id[table_id.rfind('_')+1:] + str(gt_row['target_column'])
        col = table_df[col_name]
        
        if pd.isnull(col).sum() > 0: #Base f1 = 0.31245450119322055
#             print(f'col of type {col_type} contains invalid values, file name: {fp}, col: {col_name}')
            #count returns the amount of valid values in the column, if column does
            #not contain any valid values, there is no point in trying to predict/use the column
            if col.count() == 0: # 0.35952832125142853
                continue
#             col.dropna(inplace=True) #drop the rows which do not contain a valid value \ 0.3201282795944755
#             col.ffill(inplace=True) #forward fill \ 0.3370909309989047 \ combined bfill, ffil 0.3201282795944755
#             col.bfill(inplace=True) #backward fill \ 0.33118561744140035
#             col.fillna(col.mode()[0], inplace=True) # 0.3200851212681554
#             col.fillna(col.mean(), inplace=True) #Does obviously not work for non numeric values
#             col.fillna(col.median(), inplace=True) #Does obviously not work for non numeric values
#             print(col)
                
        if (col_type == 'state'):
            print(col)
        columns.append(list(map(str, col.to_list()))) #columns need to be converted to list of strings for preprocessing later on
        col_types.append(col_type)
        col_ids.append(col_id)
        
        
#         print(list(map(str, col.to_list())))


    

0     Static
1     Static
2     Static
3     Static
4     Static
5     Static
6    Dynamic
7     Static
8    Dynamic
9    Dynamic
Name: col2, dtype: object
0     Static
1     Static
2     Static
3     Static
4     Static
5     Static
6     Static
7    Dynamic
8    Dynamic
Name: col2, dtype: object
0      Static
1      Static
2      Static
3      Static
4     Dynamic
5      Static
6      Static
7      Static
8      Static
9      Static
10    Dynamic
Name: col2, dtype: object
0     Static
1     Static
2     Static
3     Static
4    Dynamic
5    Dynamic
6     Static
7     Static
8     Static
9    Dynamic
Name: col2, dtype: object
0      Static
1      Static
2      Static
3      Static
4     Dynamic
5      Static
6      Static
7      Static
8      Static
9      Static
10     Static
11     Static
12     Static
13     Static
14     Static
15     Static
16     Static
17    Dynamic
18     Static
19    Dynamic
Name: col2, dtype: object
0     Static
1     Static
2     Static
3     Static
4     S

## Feature extraction

### Init feature extraction models

In [13]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:04.438704 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:07.068309 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.287040 seconds.


[nltk_data] Downloading package punkt to /home/senn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/senn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
data = pd.Series(
    [
        *columns #expand/unpack the list
    ],
    name="values"
)

In [15]:
data.head(5)

0    [Assets, Assets, Assets, Assets, AuthResult, A...
1    [comment, comment, comment, comment, comment, ...
2    [comment, comment, comment, comment, comment, ...
3    [Hemigrapha, astericus, asteriscus, atlantica,...
4    [genus, species, species, species, species, sp...
Name: values, dtype: object

In [16]:
extract_features(
    join(path, "processed/temporary.csv"),
    data
)
feature_vectors = pd.read_csv(join(path, "processed/temporary.csv"), dtype=np.float32)

Extracting Features:   0%|          | 0/813 [00:00<?, ?it/s]

Exporting 1588 column features


Extracting Features: 100%|██████████| 813/813 [00:04<00:00, 197.76it/s]


In [17]:
# feature_vectors.head()

### Initialize Sherlock

In [18]:
model = SherlockModel();
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

W0315 17:39:30.884222 139794286769984 deprecation.py:506] From /home/senn/virtualenvs/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0315 17:39:30.885131 139794286769984 deprecation.py:506] From /home/senn/virtualenvs/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0315 17:39:30.887143 139794286769984 deprecation.py:506] From /home/senn/virtualenvs/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling VarianceScaling._

In [19]:
predicted_labels = model.predict(feature_vectors, "sherlock")

In [20]:
# predicted_labels

## Analysing results

In [21]:
size = len(predicted_labels)
print(f'prediction count {size}')


# Should be fully deterministic too.
f1_score(col_types[:size], predicted_labels[:size], average="weighted")

prediction count 813


0.35952832125142853

### All Scores

In [22]:
print(classification_report(col_types[:size], predicted_labels[:size], digits=4))

                precision    recall  f1-score   support

       address     0.0080    1.0000    0.0159         1
           age     0.0000    0.0000    0.0000         1
         album     0.0000    0.0000    0.0000         0
          area     0.0000    0.0000    0.0000         0
        artist     0.0000    0.0000    0.0000         0
      capacity     0.0000    0.0000    0.0000         1
      category     0.0500    0.2000    0.0800         5
          city     0.8333    1.0000    0.9091         5
         class     0.2353    0.0625    0.0988        64
classification     0.2500    1.0000    0.4000         1
          club     0.0000    0.0000    0.0000         0
          code     0.8000    0.2353    0.3636        17
    collection     0.0000    0.0000    0.0000         1
       command     0.0000    0.0000    0.0000         0
       company     0.3333    1.0000    0.5000         2
     component     0.0000    0.0000    0.0000         3
       country     0.7500    0.7500    0.7500  

/home/senn/virtualenvs/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/v

In [23]:
model_id = "sherlock"

classes = np.load(f"../model_files/classes_{model_id}.npy", allow_pickle=True)

report = classification_report(col_types, predicted_labels, output_dict=True)

class_scores = list(filter(lambda x: isinstance(x, tuple) and isinstance(x[1], dict) and 'f1-score' in x[1] and x[0] in classes, list(report.items())))

class_scores = sorted(class_scores, key=lambda item: item[1]['f1-score'], reverse=True)

/home/senn/virtualenvs/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/v

### Top 5 types

In [24]:
print(f"\t\tf1-score\tprecision\trecall\t\tsupport")

for key, value in class_scores[0:5]:
    if len(key) >= 8:
        tabs = '\t' * 1
    else:
        tabs = '\t' * 2

    print(f"{key}{tabs}{value['f1-score']:.3f}\t\t{value['precision']:.3f}\t\t{value['recall']:.3f}\t\t{value['support']}")

		f1-score	precision	recall		support
year		0.959		1.000		0.922		102
city		0.909		0.833		1.000		5
country		0.750		0.750		0.750		4
type		0.733		0.917		0.611		144
status		0.625		0.588		0.667		15


### Bottom 5 types

In [25]:
print(f"\t\tf1-score\tprecision\trecall\t\tsupport")

for key, value in class_scores[len(class_scores)-5:len(class_scores)]:
    if len(key) >= 8:
        tabs = '\t' * 1
    else:
        tabs = '\t' * 2

    print(f"{key}{tabs}{value['f1-score']:.3f}\t\t{value['precision']:.3f}\t\t{value['recall']:.3f}\t\t{value['support']}")

		f1-score	precision	recall		support
service		0.000		0.000		0.000		0
sex		0.000		0.000		0.000		0
state		0.000		0.000		0.000		20
symbol		0.000		0.000		0.000		0
team		0.000		0.000		0.000		2


## Looking at the data

In [26]:
type_freq_df = pd.DataFrame(col_types, columns=['type'])
type_freq_df = pd.DataFrame(type_freq_df['type'].value_counts())
type_freq_df.columns = ['count']
type_freq_df.index.name = 'type'

In [27]:
alt.Chart(type_freq_df.reset_index()).mark_bar(size=8).encode(
    x = alt.X('type:O',
              title = 'Semantic Types',
              sort=alt.EncodingSortField(
                field="count",  
                order="descending")),
    y = alt.Y('count', title='Number of Samples')    
).properties(width=600,height=200)

alt.Chart(...)